In [5]:
import os
import sys

# Get the absolute path of 'src'
sys.path.append(os.path.abspath("../src"))

# Ensure this points to the correct JSON key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../src/config/manga-recommender-system-d0a04ad1ff87.json"

# Verify authentication
from google.cloud import bigquery

client = bigquery.Client()
print(f"Authenticated project: {client.project}")

Authenticated project: manga-recommender-system


In [8]:
from utils.bigquery_utils import load_data_from_bigquery

In [9]:
# Load data from BigQuery
df = load_data_from_bigquery(dataset_id='manga_dataset', table_id='manga_collection_raw')

2025-02-21 17:03:15.236 | INFO     | utils.bigquery_utils:load_data_from_bigquery:46 - Successfully loaded 12500 rows from manga-recommender-system.manga_dataset.manga_collection_raw


In [12]:
df

,ID,Title Romaji,Title English,Genres,Description,Average Score,Popularity,Status,Chapters,Volumes,Cover Image,Tags,Start Date,End Date,Is Adult,Ranking,Relations,Mangaka,Reviews
0,30002,Berserk,Berserk,"Action, Adventure, Drama, Fantasy, Horror, Psy...","His name is Guts, the Black Swordsman, a feare...",93.0,199722,RELEASING,NaN,NaN,https://s4.anilist.co/file/anilistcdn/media/ma...,"Tragedy, Seinen, Revenge, Male Protagonist, De...",1989.0,NaN,False,"1 (RATED - MANGA), 5 (POPULAR - MANGA)","Kenpuu Denki Berserk (ADAPTATION), Berserk: Ou...","Kentarou Miura, Kouji Mori",85 - This would be the first time i try tackli...
1,30007,Hajime no Ippo,Hajime no Ippo: Fighting Spirit!,"Action, Comedy, Drama, Sports",Makunouchi Ippo has been bullied his entire li...,87.0,28678,RELEASING,NaN,NaN,https://s4.anilist.co/file/anilistcdn/media/ma...,"Boxing, Shounen, Male Protagonist, Primarily M...",1989.0,NaN,False,"31 (RATED - MANGA), 192 (POPULAR - MANGA)","Hajime no Ippo: Champion Road (ADAPTATION), Ha...","George Morikawa, Kentarou Miura, Yuuji Ninomiya","100 - Hajime no Ippo, the best Sports manga of..."
2,30013,ONE PIECE,One Piece,"Action, Adventure, Comedy, Fantasy","As a child, Monkey D. Luffy was inspired to be...",91.0,191720,RELEASING,NaN,NaN,https://s4.anilist.co/file/anilistcdn/media/ma...,"Pirates, Travel, Found Family, Shounen, Ensemb...",1997.0,NaN,False,"4 (RATED - MANGA), 6 (POPULAR - MANGA), 1 (RAT...",ONE PIECE: Episode of Nami - Koukaishi no Nami...,"Eiichirou Oda, Yuusaku Shibata, Tatsuma Eijiri...",100 - One Piece is THE Phenomenal Piece of Fic...
3,30024,D.Gray-man,D.Gray-man,"Action, Adventure, Fantasy","A born exorcist, Walker's primary anti-akuma w...",81.0,35336,RELEASING,NaN,NaN,https://s4.anilist.co/file/anilistcdn/media/ma...,"Demons, Amnesia, Tragedy, Found Family, Super ...",2004.0,NaN,False,"265 (RATED - MANGA), 138 (POPULAR - MANGA), 3 ...","D.Gray-man (ADAPTATION), Zone (ALTERNATIVE), D...","Katsura Hoshino, Kenichi Kondou, Nichou-me no ...","100 - Never stop, keep on walking, 95 - ""I swe..."
4,30026,HUNTER×HUNTER,Hunter x Hunter,"Action, Adventure, Comedy, Fantasy","Gon might be a country boy, but he has high as...",88.0,89451,RELEASING,NaN,NaN,https://s4.anilist.co/file/anilistcdn/media/ma...,"Shounen, Super Power, Cultivation, Ensemble Ca...",1998.0,NaN,False,"22 (RATED - MANGA), 24 (POPULAR - MANGA), 2 (R...","HUNTER×HUNTER (ADAPTATION), HUNTER×HUNTER OVA ...","Yoshihiro Togashi, Naoko Takeuchi, Katsunori M...","91 - A Review of Mid x Mid, 70 - Far From Perf..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,31605,Shoukoku no Altair,Altair: A Record of Battles,"Adventure, Drama, Fantasy",For generations the Torqye Pashalu and the Bal...,75.0,4881,FINISHED,165.0,27.0,https://s4.anilist.co/file/anilistcdn/media/ma...,"War, Historical, Foreign, Politics, Military, ...",2007.0,2023.0,False,"35 (RATED - MANGA), 41 (POPULAR - MANGA)","Shoukoku no Altair (ADAPTATION), Anastasia no ...",Kotono Katou,None
12496,44997,Tiara,None,"Drama, Fantasy, Romance",Prashian is an aristocratic line gifted with 5...,NaN,225,FINISHED,253.0,35.0,https://s4.anilist.co/file/anilistcdn/media/ma...,"Female Protagonist, Full Color",2009.0,2023.0,False,None,None,None,None
12497,35685,Hakkenden,Hakkenden,"Action, Fantasy, Supernatural","Five years ago, almost every villager of Ootsu...",64.0,789,RELEASING,NaN,NaN,https://s4.anilist.co/file/anilistcdn/media/ma...,"Shoujo, Male Protagonist",2005.0,2024.0,False,None,Hakkenden: Touhou Hakken Ibun (ADAPTATION),Miyuki Abe,None
12498,41854,Mutsu Enmei-ryuu Gaiden: Shura no Toki,None,"Action, Drama",Legends tell of an invincible martial art know...,NaN,142,FINISHED,55.0,22.0,https://s4.anilist.co/file/anilistcdn/media/ma...,"Shounen, Historical",1989.0,2024.0,False,None,Mutsu Enmei Ryuu Gaiden: Shura no Toki (ADAPTA...,None,None
